In [1]:
# !pip3 install nest_asyncio
import nest_asyncio

nest_asyncio.apply()

In [2]:
import asyncio
import random


class Client:
    __salario_liquido: float = 0
    __salario_bruto: float = 0
    __debito: float = 0
    
    
    def __init__(self, nome: str, cpf = str):
        self.__nome = nome
        self.__cpf = cpf
        loop = asyncio.get_event_loop()
        self.__salario_bruto, self.__debito = loop.run_until_complete(
            asyncio.gather(self._get_salario(), self._get_debito())
        )
        self.salario_liquido = self.salario_bruto
        
    @property
    def nome(self):
        return self.__nome
    
    @property
    def cpf(self):
        return self.__cpf
    
    @property
    def salario_bruto(self):
        return self.__salario_bruto
    
    @property
    def debito(self):
        return self.__debito
    
    @property
    def salario_liquido(self):
        return self.__salario_liquido
    
    @salario_liquido.setter
    def salario_liquido(self, new_value: bool):
        self.__salario_liquido = new_value
         
    async def _get_salario(self):
        url = f"http://endpoint.com/getsalario/{self.cpf}"
        # simulating a request to another api
        # request.get(url + f"/{self.cpf}")
        await asyncio.sleep(.3)
        
        random.seed(900)
        return 4000.00 if self.cpf == "123.456.789-00" else float(random.randint(1000, 8000))
    
    async def _get_debito(self):
        url = f"http://endpoint.com/getdebito/{self.cpf}"
        # simulating a request to another api
        # request.get(url + f"/{self.cpf}")
        await asyncio.sleep(.3)
        
        random.seed(900)
        return 100.00 if self.cpf == "123.456.789-00" else float(random.randint(0, 200))
        
        
    def __repr__(self):
        return (
            f"Nome: {self.nome}\n"
            f"Salario bruto: {self.__salario_bruto}\n"
            f"Débito: {self.debito}\n"
            f"Salario_liquido: {round(self.salario_liquido, 2)}"
        )
    
ze = Client("Seu Zé", "123.456.789-00")
ze

Nome: Seu Zé
Salario bruto: 4000.0
Débito: 100.0
Salario_liquido: 4000.0

In [3]:
from abc import ABC, abstractmethod


class Interface:
    @abstractmethod
    def calculate(self, client: Client):
        pass
    
    
class INSS(Interface):
    def __init__(self, taxa=0.08):
        self.taxa = taxa
        
    def calculate(self, client: Client):
        client.salario_liquido -= (client.salario_liquido * self.taxa)
        
        
class IR(Interface):
    def __init__(self, taxa=0.06):
        self.taxa = taxa
        
    def calculate(self, client: Client):
        client.salario_liquido -= (client.salario_liquido * self.taxa)
        
        
class PagarDebito(Interface):
    def calculate(self, client: Client):
        client.salario_liquido -= client.debito
        
INSS().calculate(ze)
print(ze, end="\n\n")

IR().calculate(ze)
print(ze, end="\n\n")

PagarDebito().calculate(ze)
print(ze, end="\n\n")

Nome: Seu Zé
Salario bruto: 4000.0
Débito: 100.0
Salario_liquido: 3680.0

Nome: Seu Zé
Salario bruto: 4000.0
Débito: 100.0
Salario_liquido: 3459.2

Nome: Seu Zé
Salario bruto: 4000.0
Débito: 100.0
Salario_liquido: 3359.2



# Transformando em pipeline

In [4]:
class Calculator:
    def calcular(self, client, strategy):
        strategy.calculate(client)

In [5]:
chiquinha = Client("Chiquinha", cpf="123.456.789-01")
chiquinha

Nome: Chiquinha
Salario bruto: 6112.0
Débito: 159.0
Salario_liquido: 6112.0

In [6]:
calculator = Calculator()

for policy in [INSS(), IR(), PagarDebito()]:
    calculator.calcular(chiquinha, policy)

chiquinha

Nome: Chiquinha
Salario bruto: 6112.0
Débito: 159.0
Salario_liquido: 5126.66